In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [6]:
#export
from exp.nb_02 import *

In [3]:
path= Path('../PCImages')
path2fm= '../FeatureMatsMerged/TheGreatCollection.txt'
path2colnames= 'FeatureMatIndex.txt'

In [4]:
fm= fm_from_txt(path2fm, path2colnames)
fm.shape

(56657, 105)

In [27]:
fm.head()

,id,date,time,label1,label2,label3,ind,wallcrash,temp,LP,...,err_peKV_ElRel_2s,E_aKV_ElRel_2s,eta_aKV_ElRel_2s,Act_aKV_ElRel_2s,err_aKV_ElRel_2s,E_aeKV_ElRel_2s,E2_aeKV_ElRel_2s,eta_aeKV_ElRel_2s,Act_aeKV_ElRel_2s,err_aeKV_ElRel_2s
0,180322140137,180322.0,140137.0,0,100.0,0.0,1.0,0.000000,18.00,875.0,...,0.000840,34.801664,33.466240,3.135231e-05,0.000837,3.518287e+01,1091.754900,36.509799,0.000338,0.000819
1,180322140233,180322.0,140233.0,0,100.0,0.0,2.0,0.000000,18.01,875.0,...,0.000397,29.482185,20.546622,2.180137e-07,0.002530,1.576053e-07,64.210986,111.861860,0.000031,0.000343
2,180322140601,180322.0,140601.0,0,100.0,0.0,3.0,38.956936,17.98,875.0,...,0.028127,17.579015,3.970008,1.079131e+00,0.012464,1.876191e+01,210.278410,4.268777,1.056558,0.012754
3,180322140713,180322.0,140713.0,0,100.0,0.0,4.0,0.000000,17.99,875.0,...,0.000750,62.245363,39.791365,2.516519e-01,0.000616,5.171595e+01,993.694250,50.203118,0.320308,0.000632
4,180322140852,180322.0,140852.0,0,100.0,0.0,5.0,0.000000,17.99,875.0,...,0.002178,28.424611,9.381201,2.280459e-08,0.001762,2.870110e+01,1567.120300,9.746181,0.004185,0.001797


In [77]:
cols= ['id',
 'date',
 'time',
 'label1',
 'label2',
 'label3',
 'ind',
 'wallcrash',
 'temp',
 'LP',
 'HL',
 't_stretch',
 't_relax',
 'framecut',
 'fps',
 'medium',
 'passage',
 'ms_ch1',
 'mf_ch1',
 'mf_real_ch1',
 'cv_nn_ch1',
 'cvn_ch1',
 'ms_ch2',
 'mf_ch2',
 'mf_real_ch2',
 'cv_nn_ch2',
 'cvn_ch2']

In [78]:
fm_filtered= fm.drop(columns=cols)
fm_filtered.shape

(56657, 78)

In [35]:
img_name= get_filtered_files(path, img_ids=fm.id, labels=fm.label1)[0] 
img_name

PosixPath('../PCImages/171220_170148_stretch.png')

In [37]:
img_id, phase = split_fn(img_name)
img_id, phase

('171220170148', 'stretch')

In [74]:
r=np.array(fm.loc[fm['id'].isin(['171220170148'])]).flatten()

In [79]:
def get_row():
    pass

In [7]:
ds= CellDataSet(path, fm, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, models.resnet34(pretrained=True))
learn.fit(1, bs=16, lr=1e-5)

0 tensor(1.2644, device='cuda:0') tensor(0.5433, device='cuda:0')


In [ ]:
class CellDataSet(Dataset):
    def __init__(self, path, fm, label = 'label1', cell_phase = None, trfms=None, random_sample=None):
        self.fm = fm
        self.y = self.fm[label]
        self.id2label= id2label(self.fm.id, self.y)
        if cell_phase is None:
            self.data_files = get_filtered_files(path, img_ids=self.fm.id, labels=self.y) 
        else: 
            self.data_files= [
                f for f in get_filtered_files(path, img_ids=self.fm.id, labels=self.y) if cell_phase in str(f)
            ]
        if random_sample is not None: self.data_files= random.sample(self.data_files, random_sample)
        self.trfms = get_trfms(trfms)
        
    def __getindex__(self, idx):
        return load_file(self.data_files[idx])

    def __len__(self):
        return len(self.data_files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name  = self.data_files[idx]
        img_id, phase = split_fn(img_name)
        label=          self.id2label[img_id]
        image =         PIL.Image.open(img_name)

        if self.trfms:
            image = self.trfms(image)

        return image, label

### Concatenate models:

In [23]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc0= Resize(350)
        self.cnn = models.resnet34(pretrained=True).cuda()
        self.cnn.conv1= nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).cuda()
        self.cnn.fc= nn.Linear(in_features=512, out_features=7).cuda()
        
        #self.fc1 = nn.Linear(20 + 10, 60)
        #self.fc2 = nn.Linear(60, 5)
        
    def forward(self, image): #, data):
        x0= self.fc0(image).cuda()
        x1 = self.cnn(x0)
        #x2 = data
        
        #x = torch.cat((x1, x2), dim=1)
        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)
        return x1
        

In [24]:
class Learner():
    def __init__(self, dataset, model):
        self.ds= dataset 
        self.img_size, self.c= self.ds[0][0].shape[0], len(np.unique(self.ds.y))
        self.model= model #get_model(model.cuda(), image_size=self.img_size, c=self.c)
        self.loss= nn.CrossEntropyLoss()
        
    def fit(self, epochs=1, bs=32, lr = 1e-5):
        opt= torch.optim.Adam(self.model.parameters(), lr=lr)
        train_dl, valid_dl= ds2dls(self.ds, bs=bs)
        for epoch in range(epochs):
            self.model.train()
            for xb,yb in train_dl:
                xb= xb.to(device)
                yb= yb.to(device)
                loss = self.loss(self.model(xb), yb)
                loss.backward()
                opt.step()
                opt.zero_grad()

            self.model.eval()
            with torch.no_grad():
                tot_loss,tot_acc = 0.,0.
                for xb,yb in valid_dl:
                    xb= xb.to(device)
                    yb= yb.to(device)
                    pred = self.model(xb)
                    pred= pred.to(device)
                    tot_loss += self.loss(pred, yb)
                    tot_acc  += accuracy(pred,yb)
            nv = len(valid_dl)
            print(epoch, tot_loss/nv, tot_acc/nv)
        #return tot_loss/nv, tot_acc/nv

In [26]:
ds= CellDataSet(path, fm, label = 'label1', cell_phase = None, 
               trfms= [CenterCrop(size=350), 
                       transforms.Grayscale(num_output_channels=1), 
                       ToFloatTensor()], 
               random_sample=1000)
learn= Learner(ds, MyModel())
learn.fit(1, bs=16, lr=1e-5)

0 tensor(1.1742, device='cuda:0') tensor(0.6154, device='cuda:0')


In [ ]:
model = MyModel()

batch_size = 2
image = torch.randn(batch_size, 3, 299, 299)
data = torch.randn(batch_size, 10)

output = model(image, data)

In [ ]:
def get_model(m, image_size, c = 7):
    m.conv1= nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).cuda()
    m.fc= nn.Linear(in_features=512, out_features=c).cuda()
    return nn.Sequential(
        Resize(image_size),
        m
    )